## Concating 

1// А ты можешь в дотасет, который не уровне городов, добавить средний возраст по городу и процент женщин из тех кто указал пол?


2// В приложении дотасет по городам, можешь склеить его и табличку? 
Можешь сделать это в стате? Для меня это более прозрачное и можно руками перепроверить

In [ ]:
import pandas as pd
from datetime import datetime

In [ ]:
full = pd.read_csv("Merged_fem_groups_with_user_chars_city_names.csv")
full.head()

/var/folders/fj/5vkqx46j2sqfv58rcgb2wynr0000gn/T/ipykernel_62055/181717371.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  full = pd.read_csv("Merged_fem_groups_with_user_chars_city_names.csv")


,id,bdate,country,facebook,facebook_name,first_name,home_town,instagram,last_name,livejournal,...,87819738,114487519,65878810,128501486,204172284,209222517,189672959,163233491,225770630,city.name
0,889,20.4.1986,"{""id"":1,""title"":""Россия""}",NaN,NaN,Елена,NaN,NaN,Вихрова,NaN,...,False,False,False,False,False,False,False,False,False,Saint Petersburg
1,1097,29.9,"{""id"":1,""title"":""Россия""}",NaN,NaN,Ксения,Ленинград,kskszaza,Захарова,NaN,...,False,False,False,False,False,False,False,False,False,Saint Petersburg
2,1159,22.5,"{""id"":1,""title"":""Россия""}",NaN,NaN,Анастасия,NaN,NaN,Акела,NaN,...,False,False,False,False,False,False,False,False,False,Saint Petersburg
3,1331,11.2,"{""id"":1,""title"":""Россия""}",NaN,NaN,Елена,NaN,NaN,Каменская,NaN,...,False,False,False,False,False,False,False,False,False,Saint Petersburg
4,1632,29.3,"{""id"":1,""title"":""Россия""}",NaN,NaN,Андрей,Санкт-Петербург,andreytrusov,Трусов,NaN,...,False,False,False,False,False,False,False,False,False,Saint Petersburg


In [ ]:
# Parameters
current_year = datetime.now().year
city_column = 'city.name'  # Name of the city column
bdate_column = 'bdate'  # Column for birthdate
sex_column = 'sex'  # Column for gender (1 = female, 2 = male)
group_columns = full.columns[full.columns.str.isdigit()]  # Numeric group IDs

### Bdate

#### VK API

In [ ]:
import vk_api
import pandas as pd
import re
from datetime import datetime

# Read the access token
with open('/Users/olgafoorgy/Documents/Documents - Ноутбук — Ольга/groups/TOKEN VK') as f:
    access_token = f.read().strip()

# Initialize VK session
vk_session = vk_api.VkApi(token=access_token)
vk = vk_session.get_api()

#### Begin working on incomplete bday

In [ ]:


# Parameters
current_year = datetime.now().year

# Function to calculate age
def calculate_age(bdate):
    try:
        parts = bdate.split('.')
        if len(parts) == 3:  # Format: dd.mm.yyyy
            year = int(parts[2])
            return current_year - year
        elif len(parts) == 2:  # Format: dd.mm
            return None  # Missing year
        else:
            return None  # Invalid format
    except (ValueError, AttributeError):
        return None  # Handle missing or invalid dates

# Extract user IDs with missing or incomplete bdate
missing_bdate_ids = full[full['bdate'].str.len() < 8]['id'].tolist()  # Assuming `id` column contains VK IDs

# Fetch updated user information from VK API
fetched_data = []
batch_size = 100  # VK API allows up to 100 IDs per request
for i in range(0, len(missing_bdate_ids), batch_size):
    ids_batch = missing_bdate_ids[i:i + batch_size]
    response = vk.users.get(user_ids=ids_batch, fields='bdate')
    fetched_data.extend(response)

# Update bdate values if fetched bdate is more complete
fetched_bdates = {str(user['id']): user.get('bdate', '') for user in fetched_data}
for index, row in full.iterrows():
    user_id = str(row['id'])
    current_bdate = row['bdate']
    fetched_bdate = fetched_bdates.get(user_id, '')

    # Replace if fetched bdate contains a year and is more complete
    if len(fetched_bdate.split('.')) == 3 and len(current_bdate.split('.')) < 3:
        full.at[index, 'bdate'] = fetched_bdate

# Recalculate the 'age' column
full['age'] = full['bdate'].apply(calculate_age)

In [ ]:
# Ensure group columns are boolean
full[group_columns] = full[group_columns].astype(bool)

# Add gender boolean (is_female = 1 if female)
full['is_female'] = full[sex_column] == 1

# Group by city
grouped = full.groupby(city_column)



In [ ]:
# Calculate total subscribers per city (across all groups)
total_subs = grouped[group_columns].sum().sum(axis=1)  # Sum over groups for each city

# Calculate total women subscribers per city
total_women_subs = grouped.apply(
    lambda x: x[group_columns].multiply(x['is_female'], axis=0).sum().sum()
)

# Calculate average age (only valid ages)
def average_age_per_group(x):
    valid_age_mask = x['age'].notnull()  # Only consider rows with valid ages
    age_weighted_sum = x.loc[valid_age_mask, group_columns].multiply(
        x.loc[valid_age_mask, 'age'], axis=0
    ).sum()
    group_sums = x.loc[valid_age_mask, group_columns].sum()
    return age_weighted_sum / group_sums

city_age = grouped.apply(average_age_per_group)

# Calculate percentage of women subscribers
city_gender = grouped.apply(
    lambda x: x[group_columns].multiply(x['is_female'], axis=0).sum() / x[group_columns].sum() * 100
)

# Combine with existing statistics
city_stats = grouped[group_columns].sum()  # Total subscribers per group in each city
city_stats['avg_age'] = city_age.mean(axis=1)  # Average age
city_stats['avg_women_percent'] = city_gender.mean(axis=1)  # Average percentage of women
city_stats['total_subs'] = total_subs  # Total subscriptions
city_stats['total_women_subs'] = total_women_subs  # Total women subscribers



/var/folders/fj/5vkqx46j2sqfv58rcgb2wynr0000gn/T/ipykernel_62055/332991624.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  total_women_subs = grouped.apply(
/var/folders/fj/5vkqx46j2sqfv58rcgb2wynr0000gn/T/ipykernel_62055/332991624.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  city_age = grouped.apply(average_age_per_group)
/var/folders/fj/5vkqx46j2sqfv58rcgb2wynr0000gn/T/ipykernel_62055/33299162

           81914161  190884646  80728963  153276634  96917976  204732875  \
city.name                                                                  
Aarschot          0          0         0          0         1          0   
Abakan            1          0         1          0         8          0   
Abaza             0          0         0          0         1          0   
Aberdeen          0          0         0          0         1          0   
Abinsk            0          0         0          0         0          0   

           198043329  159437367  74599974  195772623  ...  128501486  \
city.name                                             ...              
Aarschot           0          0         0          0  ...          0   
Abakan             0          1         3          0  ...          0   
Abaza              0          0         0          0  ...          0   
Aberdeen           0          0         0          0  ...          0   
Abinsk             0          0    

In [ ]:
# Save or display the result
city_stats.to_csv("city_group_subscribers_with_extended_stats.csv")
city_stats.to_excel("city_group_subscribers_with_extended_stats.xlsx")
print(city_stats.head())

### Group names

In [ ]:
# Load your existing DataFrame
merged = pd.read_csv("city_group_subscribers_with_extended_stats.csv")

# Extract numeric group IDs from the DataFrame columns
group_ids = [col for col in merged.columns if re.match(r'^\d+$', col)]

# Fetch group names using the VK API
group_info = vk.groups.getById(group_ids=group_ids, fields="name")

# Create a mapping of group IDs to group names
group_names = {str(item['id']): item['name'] for item in group_info}

# Rename columns in the DataFrame
merged.rename(columns=group_names, inplace=True)

# Save the updated DataFrame to a new CSV or display it
merged.to_csv("city_group_subscribers_with_extended_stats.csv", index=False)
merged.to_excel("city_group_subscribers_with_extended_stats.xlsx")

print(merged.head())

  city.name  Живи с подругой!  \
0  Aarschot                 0   
1    Abakan                 1   
2     Abaza                 0   
3  Aberdeen                 0   
4    Abinsk                 0   

   Уютное гнездышко / поиск жилья и соседей /Резерв  ALKOSHMOT  alkoshmot  \
0                                                 0          0          0   
1                                                 0          1          0   
2                                                 0          0          0   
3                                                 0          0          0   
4                                                 0          0          0   

   Феминизм: наглядно.  Феминизм: наглядно  Феминизм: наглядно  \
0                    1                   0                   0   
1                    8                   0                   0   
2                    1                   0                   0   
3                    1                   0                   0   
4       

## Заплатка чтобы объединить группы по полу

In [ ]:
# After renaming columns, print the columns to debug
print("Updated Columns:", merged.columns)

# Create a list of group names
group_names_list = list(group_names.values())
print("Group Names List:", group_names_list)

# Ensure that the group names exist in the columns before proceeding
valid_group_names = [name for name in group_names_list if name in merged.columns]

# Group users by city and calculate subscriber counts for each valid group
city_group_counts = merged.groupby('sex')[valid_group_names].sum()

# Resulting table
result_table = city_group_counts

# Optionally, save the result to a CSV
result_table.to_csv('sex_group_subscribers.csv')

# Display the result
print(result_table)
